In [ ]:
#https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py

#https://sigmoidal.io/chatbots-for-b2c-and-deep-learning/

In [ ]:
#dump ke mongo
#mongoimport --db chatbot_ta --collection electronics --file qa_electronics.json

#to open mongo : sudo service mongodb start
#mongo

In [49]:
from pymongo import MongoClient
import pandas as pd
import numpy as npr
pd.set_option('display.max_colwidth',-1) 
client = MongoClient('localhost', 27017) 
db=client.get_database('ta_chatbot')
collection=db.get_collection('electronics')

In [50]:
cursor=collection.find()
data = pd.DataFrame(list(collection.find())) 

data.head()

,_id,answer,answerTime,answerType,asin,question,questionType,unixTime
0,5b445f49bc4775cb089d955f,Yes this fits both the nook color and the same-shaped nook tablet,"Dec 27, 2013",Y,0594033926,Is this cover the one that fits the old nook color? Which I believe is 8x5.,yes/no,1.388131e+09
1,5b445f49bc4775cb089d9560,No. The nook color or color tablet,"Jan 5, 2015",N,0594033926,Does it fit Nook GlowLight?,yes/no,1.420445e+09
2,5b445f49bc4775cb089d9561,yes,17 days ago,Y,0594033926,Will this fit a Nook Color that's 5 x 8?,yes/no,NaN
3,5b445f49bc4775cb089d9562,"I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close.",2 days ago,NaN,0594033926,Would it fit Nook 1st Edition? 4.9in x 7.7in ?,open-ended,NaN
4,5b445f49bc4775cb089d9563,"No, the tab is smaller than the 'color'","Feb 10, 2015",N,0594033926,will this fit the Samsung Galaxy Tab 4 Nook 10.1,yes/no,1.423555e+09


In [68]:
data_ques=data[['question','answer']][:15000]
data_ques[:15000]

,question,answer
0,Is this cover the one that fits the old nook color? Which I believe is 8x5.,Yes this fits both the nook color and the same-shaped nook tablet
1,Does it fit Nook GlowLight?,No. The nook color or color tablet
2,Will this fit a Nook Color that's 5 x 8?,yes
3,Would it fit Nook 1st Edition? 4.9in x 7.7in ?,"I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close."
4,will this fit the Samsung Galaxy Tab 4 Nook 10.1,"No, the tab is smaller than the 'color'"
5,does it have a flip stand?,"No, there is not a flip stand. It has a pocket in the front flap. It is a very nice cover."
6,does this have a flip stand,"Hi, no it doesn't"
7,also fits the HD+?,It should. They are the same size and the charging port is in the same place.
8,Does it have 2 positions for the reader? Horizontal/vertical Thank You KWOD,Yes
9,how far out does the arm extend?,18 inches on our TV.


In [70]:
# Preproses:
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
porter_stemmer = PorterStemmer()

## Remove Punctuation

In [71]:
import re
def removePunctuation(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
   
    return text

# Clean the data
data_removePunctuation = []
coba = pd.DataFrame(data_ques)    
for line in coba.itertuples():
    data_removePunctuation.append(removePunctuation(line[1]))

In [76]:
# data_removePunctuation
data_normal_remPunctuation = pd.DataFrame(data_removePunctuation)
data_normal_remPunctuation = data_normal_remPunctuation.rename(columns = {0:'question'})
data_normal_remPunctuation.head(15000)

,question
0,Is this cover the one that fits the old nook color Which I believe is 8x5
1,Does it fit Nook GlowLight
2,Will this fit a Nook Color that's 5 x 8
3,Would it fit Nook 1st Edition 49in x 77in
4,will this fit the Samsung Galaxy Tab 4 Nook 101
5,does it have a flip stand
6,does this have a flip stand
7,also fits the HD
8,Does it have 2 positions for the reader Horizontalvertical Thank You KWOD
9,how far out does the arm extend


## Case Folding

In [77]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
porter_stemmer = PorterStemmer()

def casefolding(corpus):
   
    print ("proses lowercase")

    for item in range(len(corpus)):
        text = str.lower(corpus['question'].iloc[item])
        
        corpus['question'].iloc[item] = text
    return corpus

In [78]:
data_normal_caseFolding = casefolding(data_normal_remPunctuation)
print ('jumlah data:',len(data_normal_caseFolding))

proses lowercase
jumlah data: 15000


In [80]:
data_normal_caseFolding.head(15000)

,question
0,is this cover the one that fits the old nook color which i believe is 8x5
1,does it fit nook glowlight
2,will this fit a nook color that's 5 x 8
3,would it fit nook 1st edition 49in x 77in
4,will this fit the samsung galaxy tab 4 nook 101
5,does it have a flip stand
6,does this have a flip stand
7,also fits the hd
8,does it have 2 positions for the reader horizontalvertical thank you kwod
9,how far out does the arm extend


## Normalization (Contraction)

In [84]:
contractions = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

def contraction_proc(text):
    for word in text.split():
        if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])
            text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text


# Clean the data
data_lemma = [15000]
coba = pd.DataFrame(data_normal_caseFolding)    
for line in coba.itertuples():
    data_lemma.append(contraction_proc(line[1]))

    
data_lemma

[15000,
 'is this cover the one that fits the old nook color which i believe is 8x5',
 'does it fit nook glowlight',
 'will this fit a nook color that has  that is 5 x 8',
 'would it fit nook 1st edition 49in x 77in ',
 'will this fit the samsung galaxy tab 4 nook 101',
 'does it have a flip stand',
 'does this have a flip stand',
 'also fits the hd',
 'does it have 2 positions for the reader horizontalvertical thank you kwod',
 'how far out does the arm extend',
 'my vizio has 200 ht x 600 width mounting holes will this mount handle that',
 'will it fit vizio tvs',
 'my tv has a vesa 200x200 bolt pattern and i need it to extend to 24 inches with the extension will this wall mount fit my bolt pattern',
 'can you mount a sound bar using the external stick arms vertically hanging below the lcd monitor 27 inch',
 'can i use it on a tv with vesa 100 x 100',
 'does it work well with 39 inch magnavox tv',
 'is there a closure mechanism bands magnetic etc',
 'can it be used for vizio 39 class

In [87]:
data_normal_contraction= pd.DataFrame(data_lemma)
data_normal_contraction = data_normal_contraction.rename(columns = {0:'question'})
data_normal_contraction.head(15000)

,question
0,15000
1,is this cover the one that fits the old nook color which i believe is 8x5
2,does it fit nook glowlight
3,will this fit a nook color that has that is 5 x 8
4,would it fit nook 1st edition 49in x 77in
5,will this fit the samsung galaxy tab 4 nook 101
6,does it have a flip stand
7,does this have a flip stand
8,also fits the hd
9,does it have 2 positions for the reader horizontalvertical thank you kwod


## Stopwords Removal

In [63]:
# Preproses:
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
porter_stemmer = PorterStemmer()

def stopwordsRemoval(corpus):
    print ("proses stopwords removal")
    cachedStopWords = set(stopwords.words("english"))
    for i in range(len(corpus)):
        sent=corpus['question'].iloc[i]
        kt=" ".join([word for word in sent.split() if word not in cachedStopWords])
        corpus['question'].iloc[i]=kt

    return corpus

In [64]:
data_normal_stopwordsRemoval = stopwordsRemoval(data_normal_contraction)
print ('jumlah data:',len(data_normal_stopwordsRemoval))

proses stopwords removal
jumlah data: 1000


In [65]:
data_normal_stopwordsRemoval.head()

,question
0,cover one fits old nook color believe 8x5
1,fit nook glowlight
2,fit nook color 5 x 8
3,would fit nook 1st edition 49in x 77in
4,fit samsung galaxy tab 4 nook 101


## create to file

In [88]:
file = open('_electronics_data_15000_qa_fix_for_training.txt','w')
for i in range(len(data_normal_stopwordsRemoval['question'])):
    q = data_normal_stopwordsRemoval['question']
    a = data_ques['answer']
    print (q[i]+'\t'+a[i])
    file.write(q[i]+'\t'+a[i])
    file.write('\n')
file.close()

cover one fits old nook color believe 8x5	Yes this fits both the nook color and the same-shaped nook tablet
fit nook glowlight	No. The nook color or color tablet
fit nook color 5 x 8	yes
would fit nook 1st edition 49in x 77in	I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close.
fit samsung galaxy tab 4 nook 101	No, the tab is smaller than the 'color'
flip stand	No, there is not a flip stand. It has a pocket in the front flap. It is a very nice cover.
flip stand	Hi, no it doesn't
also fits hd	It should. They are the same size and the charging port is in the same place.
2 positions reader horizontalvertical thank kwod	Yes
far arm extend	18 inches on our TV.
vizio 200 ht x 600 width mounting holes mount handle	I'm sorry mine is mounted already so I could not measure for you but as long as it's within the specified size it should fit just fine I recommend
fit vizio tvs	i have a 42" vizio and it fit just fine
tv vesa 200x200 bolt p

# Next Step

In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

C:\Users\ASUS\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras import backend as K
import tensorflow as tf
with K.tf.device('/gpu:4'):
    config = tf.ConfigProto(intra_op_parallelism_threads=8,\
           inter_op_parallelism_threads=16, allow_soft_placement=True,\
           device_count = {'CPU' : 4, 'GPU' : 4})
    session = tf.Session(config=config)
    K.set_session(session) 

In [3]:
batch_size = 64  # Batch size for training.
epochs = 1000  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 1000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'electronics_data_15000_qa_fix_for_training_update.txt'

In [4]:
# Vectorize the data.
import io
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with io.open(data_path, 'r', encoding ='utf-8') as f:
    lines = f.read().split('\n')
    for line in lines[: min(num_samples, len(lines) - 1)]:
        input_text, target_text = line.split('\t')
        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        target_text = '\t' + target_text + '\n'
        input_texts.append(input_text)
        target_texts.append(target_text)
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)
        
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 1000
Number of unique input tokens: 50
Number of unique output tokens: 93
Max sequence length for inputs: 156
Max sequence length for outputs: 1209


In [6]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])


encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
#encoder_input_data is a 3D matrix of shape(number_of_pairs, max_question_sequence_legth, number_of_question_character)

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
#decoder_input_data is a 3D matrix of shape(number_of_pairs, max_answer_sequence_legth, number_of_answer_character)

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
dtype='float32')
#decoder_target_data is the same as decoder_input_data shifted one time step ahead. This means that decoder_input_data
#    [:, t+1, :] is equals to decoder_target_data[:, t, :]
#our goal will be to train a seq2seq model to predict decoder_output_data
#using encoder_input_data and decoder_input data as our input features.



In [7]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
            

In [8]:
#https://books.google.co.id/books?id=M5RRDwAAQBAJ&pg=PA184&dq=encoder_input_data+%3D+np.zeros((len(input_texts),+max_encoder_seq_length,+num_encoder_tokens),dtype%3D%27float32%27)decoder_input_data+%3D+np.zeros((len(input_texts),+max_decoder_seq_length,+num_decoder_tokens),dtype%3D%27float32%27)decoder_target_data+%3D+np.zeros((len(input_texts),+max_decoder_seq_length,+num_decoder_tokens),dtype%3D%27float32%27)&hl=id&sa=X&ved=0ahUKEwjptojM2ILbAhVIPY8KHT0IAvIQ6AEIKjAA#v=onepage&q=data%20prep&f=false

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


In [9]:
# Start to build seq2seq network architecture.

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
# The return_state contructor argument, configuring a RNN layer to return a list where the first entry is the outputs and the next entries are the internal RNN states. This is used to recover the states of the encoder.
# The inital_state call argument, specifying the initial state(s) of a RNN. This is used to pass the encoder states to the decoder as initial states.
# The return_sequences constructor argument, configuring a RNN to return its full sequence of outputs (instead of just the last output, which the defaults behavior). This is used in the decoder.

In [10]:
from keras.models import Sequential
from keras.layers import Dense
import numpy

In [11]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [12]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [13]:
# To Training model, run this code. But, if you want to use pretrained model, dont't run it.
# Fit the model
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
model.summary()

Train on 800 samples, validate on 200 samples
Epoch 1/1000
800/800 [==============================] - 27s 34ms/step - loss: 0.0150 - acc: 0.9990 - val_loss: 0.0148 - val_acc: 0.9988
Epoch 2/1000
800/800 [==============================] - 25s 31ms/step - loss: 0.0143 - acc: 0.9990 - val_loss: 0.0147 - val_acc: 0.9988
Epoch 3/1000
800/800 [==============================] - 27s 34ms/step - loss: 0.0143 - acc: 0.9990 - val_loss: 0.0147 - val_acc: 0.9988
Epoch 4/1000
800/800 [==============================] - 26s 32ms/step - loss: 0.0143 - acc: 0.9990 - val_loss: 0.0147 - val_acc: 0.9988
Epoch 5/1000
800/800 [==============================] - 27s 34ms/step - loss: 0.0142 - acc: 0.9990 - val_loss: 0.0146 - val_acc: 0.9988
Epoch 6/1000
800/800 [==============================] - 27s 34ms/step - loss: 0.0142 - acc: 0.9990 - val_loss: 0.0145 - val_acc: 0.9988
Epoch 7/1000
800/800 [==============================] - 26s 33ms/step - loss: 0.0142 - acc: 0.9990 - val_loss: 0.0145 - val_acc: 0.9988
Ep

800/800 [==============================] - 25s 32ms/step - loss: 0.0128 - acc: 0.9991 - val_loss: 0.0132 - val_acc: 0.9990
Epoch 61/1000
800/800 [==============================] - 25s 31ms/step - loss: 0.0128 - acc: 0.9991 - val_loss: 0.0132 - val_acc: 0.9990
Epoch 62/1000
800/800 [==============================] - 25s 31ms/step - loss: 0.0128 - acc: 0.9991 - val_loss: 0.0132 - val_acc: 0.9990
Epoch 63/1000
800/800 [==============================] - 25s 31ms/step - loss: 0.0128 - acc: 0.9991 - val_loss: 0.0132 - val_acc: 0.9990
Epoch 64/1000
800/800 [==============================] - 25s 31ms/step - loss: 0.0128 - acc: 0.9991 - val_loss: 0.0132 - val_acc: 0.9990
Epoch 65/1000
800/800 [==============================] - 25s 31ms/step - loss: 0.0127 - acc: 0.9991 - val_loss: 0.0132 - val_acc: 0.9990
Epoch 66/1000
800/800 [==============================] - 25s 31ms/step - loss: 0.0127 - acc: 0.9991 - val_loss: 0.0131 - val_acc: 0.9990
Epoch 67/1000
800/800 [==============================] 

Epoch 120/1000
800/800 [==============================] - 25s 32ms/step - loss: 0.0121 - acc: 0.9993 - val_loss: 0.0130 - val_acc: 0.9990
Epoch 121/1000
800/800 [==============================] - 25s 32ms/step - loss: 0.0121 - acc: 0.9993 - val_loss: 0.0130 - val_acc: 0.9990
Epoch 122/1000
800/800 [==============================] - 26s 32ms/step - loss: 0.0121 - acc: 0.9993 - val_loss: 0.0130 - val_acc: 0.9990
Epoch 123/1000
800/800 [==============================] - 25s 32ms/step - loss: 0.0121 - acc: 0.9993 - val_loss: 0.0130 - val_acc: 0.9990
Epoch 124/1000
800/800 [==============================] - 25s 32ms/step - loss: 0.0121 - acc: 0.9993 - val_loss: 0.0130 - val_acc: 0.9990
Epoch 125/1000
800/800 [==============================] - 26s 32ms/step - loss: 0.0121 - acc: 0.9993 - val_loss: 0.0131 - val_acc: 0.9990
Epoch 126/1000
800/800 [==============================] - 26s 32ms/step - loss: 0.0121 - acc: 0.9993 - val_loss: 0.0130 - val_acc: 0.9990
Epoch 127/1000
800/800 [==========

800/800 [==============================] - 25s 32ms/step - loss: 0.0116 - acc: 0.9994 - val_loss: 0.0134 - val_acc: 0.9989
Epoch 180/1000
800/800 [==============================] - 25s 32ms/step - loss: 0.0116 - acc: 0.9994 - val_loss: 0.0134 - val_acc: 0.9989
Epoch 181/1000
800/800 [==============================] - 26s 32ms/step - loss: 0.0116 - acc: 0.9994 - val_loss: 0.0134 - val_acc: 0.9990
Epoch 182/1000
800/800 [==============================] - 27s 33ms/step - loss: 0.0116 - acc: 0.9994 - val_loss: 0.0134 - val_acc: 0.9989
Epoch 183/1000
800/800 [==============================] - 26s 32ms/step - loss: 0.0116 - acc: 0.9994 - val_loss: 0.0135 - val_acc: 0.9989
Epoch 184/1000
800/800 [==============================] - 25s 32ms/step - loss: 0.0116 - acc: 0.9995 - val_loss: 0.0134 - val_acc: 0.9989
Epoch 185/1000
800/800 [==============================] - 25s 31ms/step - loss: 0.0116 - acc: 0.9995 - val_loss: 0.0134 - val_acc: 0.9989
Epoch 186/1000
800/800 [=========================

800/800 [==============================] - 25s 32ms/step - loss: 0.0111 - acc: 0.9996 - val_loss: 0.0141 - val_acc: 0.9988
Epoch 239/1000
800/800 [==============================] - 25s 31ms/step - loss: 0.0111 - acc: 0.9996 - val_loss: 0.0141 - val_acc: 0.9988
Epoch 240/1000
800/800 [==============================] - 25s 31ms/step - loss: 0.0111 - acc: 0.9996 - val_loss: 0.0141 - val_acc: 0.9988
Epoch 241/1000
800/800 [==============================] - 26s 32ms/step - loss: 0.0111 - acc: 0.9996 - val_loss: 0.0141 - val_acc: 0.9988
Epoch 242/1000
800/800 [==============================] - 25s 31ms/step - loss: 0.0111 - acc: 0.9996 - val_loss: 0.0141 - val_acc: 0.9988
Epoch 243/1000
800/800 [==============================] - 26s 32ms/step - loss: 0.0111 - acc: 0.9996 - val_loss: 0.0141 - val_acc: 0.9988
Epoch 244/1000
800/800 [==============================] - 25s 31ms/step - loss: 0.0111 - acc: 0.9996 - val_loss: 0.0141 - val_acc: 0.9988
Epoch 245/1000
800/800 [=========================

...

800/800 [==============================] - 25s 32ms/step - loss: 0.0100 - acc: 1.0000 - val_loss: 0.0183 - val_acc: 0.9986
Epoch 711/1000
800/800 [==============================] - 25s 32ms/step - loss: 0.0100 - acc: 0.9999 - val_loss: 0.0182 - val_acc: 0.9986
Epoch 712/1000
800/800 [==============================] - 26s 32ms/step - loss: 0.0100 - acc: 1.0000 - val_loss: 0.0183 - val_acc: 0.9986
Epoch 713/1000
800/800 [==============================] - 25s 32ms/step - loss: 0.0100 - acc: 0.9999 - val_loss: 0.0182 - val_acc: 0.9986
Epoch 714/1000
800/800 [==============================] - 25s 32ms/step - loss: 0.0099 - acc: 1.0000 - val_loss: 0.0183 - val_acc: 0.9986
Epoch 715/1000
800/800 [==============================] - 26s 32ms/step - loss: 0.0101 - acc: 0.9999 - val_loss: 0.0183 - val_acc: 0.9986
Epoch 716/1000
800/800 [==============================] - 26s 32ms/step - loss: 0.0099 - acc: 1.0000 - val_loss: 0.0183 - val_acc: 0.9986
Epoch 717/1000
800/800 [=========================

800/800 [==============================] - 26s 32ms/step - loss: 0.0099 - acc: 1.0000 - val_loss: 0.0185 - val_acc: 0.9986
Epoch 770/1000
800/800 [==============================] - 29s 36ms/step - loss: 0.0100 - acc: 0.9999 - val_loss: 0.0185 - val_acc: 0.9986
Epoch 771/1000
800/800 [==============================] - 27s 33ms/step - loss: 0.0100 - acc: 0.9999 - val_loss: 0.0185 - val_acc: 0.9986
Epoch 772/1000
800/800 [==============================] - 27s 33ms/step - loss: 0.0099 - acc: 1.0000 - val_loss: 0.0185 - val_acc: 0.9986
Epoch 773/1000
800/800 [==============================] - 27s 33ms/step - loss: 0.0099 - acc: 1.0000 - val_loss: 0.0186 - val_acc: 0.9986
Epoch 774/1000
800/800 [==============================] - 27s 34ms/step - loss: 0.0100 - acc: 0.9999 - val_loss: 0.0185 - val_acc: 0.9986
Epoch 775/1000
800/800 [==============================] - 27s 34ms/step - loss: 0.0099 - acc: 1.0000 - val_loss: 0.0185 - val_acc: 0.9986
Epoch 776/1000
800/800 [=========================

800/800 [==============================] - 27s 34ms/step - loss: 0.0100 - acc: 0.9999 - val_loss: 0.0187 - val_acc: 0.9986
Epoch 829/1000
800/800 [==============================] - 28s 35ms/step - loss: 0.0099 - acc: 1.0000 - val_loss: 0.0187 - val_acc: 0.9986
Epoch 830/1000
800/800 [==============================] - 28s 35ms/step - loss: 0.0099 - acc: 1.0000 - val_loss: 0.0188 - val_acc: 0.9986
Epoch 831/1000
800/800 [==============================] - 27s 34ms/step - loss: 0.0099 - acc: 1.0000 - val_loss: 0.0188 - val_acc: 0.9986
Epoch 832/1000
800/800 [==============================] - 26s 32ms/step - loss: 0.0100 - acc: 0.9999 - val_loss: 0.0187 - val_acc: 0.9986
Epoch 833/1000
800/800 [==============================] - 26s 33ms/step - loss: 0.0099 - acc: 1.0000 - val_loss: 0.0187 - val_acc: 0.9986
Epoch 834/1000
800/800 [==============================] - 26s 33ms/step - loss: 0.0099 - acc: 1.0000 - val_loss: 0.0187 - val_acc: 0.9986
Epoch 835/1000
800/800 [=========================

800/800 [==============================] - 27s 34ms/step - loss: 0.0099 - acc: 1.0000 - val_loss: 0.0189 - val_acc: 0.9986
Epoch 888/1000
800/800 [==============================] - 27s 33ms/step - loss: 0.0110 - acc: 0.9995 - val_loss: 0.0217 - val_acc: 0.9972
Epoch 889/1000
800/800 [==============================] - 30s 37ms/step - loss: 0.0128 - acc: 0.9988 - val_loss: 0.0211 - val_acc: 0.9973
Epoch 890/1000
800/800 [==============================] - 30s 37ms/step - loss: 0.0127 - acc: 0.9988 - val_loss: 0.0208 - val_acc: 0.9973
Epoch 891/1000
800/800 [==============================] - 27s 34ms/step - loss: 0.0125 - acc: 0.9989 - val_loss: 0.0206 - val_acc: 0.9974
Epoch 892/1000
800/800 [==============================] - 26s 32ms/step - loss: 0.0124 - acc: 0.9989 - val_loss: 0.0204 - val_acc: 0.9975
Epoch 893/1000
800/800 [==============================] - 25s 31ms/step - loss: 0.0124 - acc: 0.9989 - val_loss: 0.0203 - val_acc: 0.9975
Epoch 894/1000
800/800 [=========================

800/800 [==============================] - 31s 39ms/step - loss: 0.0115 - acc: 0.9994 - val_loss: 0.0183 - val_acc: 0.9985
Epoch 947/1000
800/800 [==============================] - 28s 35ms/step - loss: 0.0115 - acc: 0.9994 - val_loss: 0.0183 - val_acc: 0.9985
Epoch 948/1000
800/800 [==============================] - 33s 41ms/step - loss: 0.0115 - acc: 0.9995 - val_loss: 0.0183 - val_acc: 0.9985
Epoch 949/1000
800/800 [==============================] - 27s 33ms/step - loss: 0.0115 - acc: 0.9995 - val_loss: 0.0182 - val_acc: 0.9985
Epoch 950/1000
800/800 [==============================] - 28s 36ms/step - loss: 0.0115 - acc: 0.9995 - val_loss: 0.0182 - val_acc: 0.9985
Epoch 951/1000
800/800 [==============================] - 29s 36ms/step - loss: 0.0115 - acc: 0.9995 - val_loss: 0.0182 - val_acc: 0.9985
Epoch 952/1000
800/800 [==============================] - 33s 42ms/step - loss: 0.0115 - acc: 0.9995 - val_loss: 0.0182 - val_acc: 0.9985
Epoch 953/1000
800/800 [=========================

In [14]:
# Run this code, if you run training model above.
model.save_weights('model_for_sidang_ta_si_03_1000d_1000e_update.h5')
print("Saved model to disk")

Saved model to disk


In [15]:
# load weights into new model
model.load_weights("model_for_sidang_ta_si_03_1000d_1000e_update.h5")
print("Loaded model from disk")

Loaded model from disk


In [16]:
history.history['acc']

[0.9989613866806031,
 0.9989614272117615,
 0.9989614343643188,
 0.9989614152908325,
 0.9989614295959472,
 0.9989614367485047,
 0.9989614152908325,
 0.9989614224433899,
 0.9989614319801331,
 0.9989614224433899,
 0.9989614272117615,
 0.998961398601532,
 0.9989614033699036,
 0.9989614295959472,
 0.9989614796638489,
 0.9989621233940125,
 0.9989625120162964,
 0.9989636397361755,
 0.9989666700363159,
 0.9989688491821289,
 0.9989730620384216,
 0.9989790368080139,
 0.998984522819519,
 0.9989874672889709,
 0.9989953660964965,
 0.998998703956604,
 0.9990026307106018,
 0.9990061593055725,
 0.9990106534957885,
 0.9990120959281922,
 0.9990191721916198,
 0.9990219545364379,
 0.9990284657478332,
 0.9990295433998108,
 0.999037013053894,
 0.9990380024909973,
 0.9990401649475098,
 0.9990447139739991,
 0.9990496730804443,
 0.9990531992912293,
 0.999056191444397,
 0.9990610933303833,
 0.9990615773200989,
 0.999067509174347,
 0.9990688705444336,
 0.9990706396102905,
 0.9990736889839172,
 0.9990787148475647

In [17]:
history.history['acc']

[0.9989613866806031,
 0.9989614272117615,
 0.9989614343643188,
 0.9989614152908325,
 0.9989614295959472,
 0.9989614367485047,
 0.9989614152908325,
 0.9989614224433899,
 0.9989614319801331,
 0.9989614224433899,
 0.9989614272117615,
 0.998961398601532,
 0.9989614033699036,
 0.9989614295959472,
 0.9989614796638489,
 0.9989621233940125,
 0.9989625120162964,
 0.9989636397361755,
 0.9989666700363159,
 0.9989688491821289,
 0.9989730620384216,
 0.9989790368080139,
 0.998984522819519,
 0.9989874672889709,
 0.9989953660964965,
 0.998998703956604,
 0.9990026307106018,
 0.9990061593055725,
 0.9990106534957885,
 0.9990120959281922,
 0.9990191721916198,
 0.9990219545364379,
 0.9990284657478332,
 0.9990295433998108,
 0.999037013053894,
 0.9990380024909973,
 0.9990401649475098,
 0.9990447139739991,
 0.9990496730804443,
 0.9990531992912293,
 0.999056191444397,
 0.9990610933303833,
 0.9990615773200989,
 0.999067509174347,
 0.9990688705444336,
 0.9990706396102905,
 0.9990736889839172,
 0.9990787148475647

In [18]:
history.history['val_acc']

[0.9988136720657349,
 0.9988136720657349,
 0.9988136720657349,
 0.9988136720657349,
 0.9988136720657349,
 0.9988136720657349,
 0.9988136720657349,
 0.9988136720657349,
 0.9988136720657349,
 0.9988136720657349,
 0.9988136720657349,
 0.9988136720657349,
 0.9988136720657349,
 0.9988136720657349,
 0.9988136720657349,
 0.9988136720657349,
 0.998816876411438,
 0.9988145947456359,
 0.9988244605064392,
 0.998833417892456,
 0.9988267302513123,
 0.9988451862335205,
 0.998858163356781,
 0.9988592052459717,
 0.9988389492034913,
 0.9988648128509522,
 0.9988753151893616,
 0.9988767838478089,
 0.9988702487945557,
 0.9988840961456299,
 0.9988845348358154,
 0.998892023563385,
 0.9988953804969788,
 0.9988975095748901,
 0.9988996577262879,
 0.9988956284523011,
 0.9989015650749207,
 0.9989010095596313,
 0.9989117503166198,
 0.9989189767837524,
 0.9989244508743286,
 0.9989269518852234,
 0.9989321255683898,
 0.9989346122741699,
 0.9989238238334656,
 0.9989399647712708,
 0.9989339971542358,
 0.99894011259078

In [19]:
np.mean(history.history['val_acc'])

0.9986751681947708

In [20]:
history.history['loss']

[0.01500117838382721,
 0.014310499057173728,
 0.014282811805605889,
 0.014334737509489059,
 0.014246433302760124,
 0.01420934833586216,
 0.014158996790647506,
 0.01418934054672718,
 0.014018850028514862,
 0.014003086425364018,
 0.013974520936608314,
 0.01393359448760748,
 0.01390835367143154,
 0.013834191858768463,
 0.013860013633966446,
 0.013722377941012382,
 0.01369545292109251,
 0.013625258058309555,
 0.013570077791810036,
 0.01357346225529909,
 0.013497400432825088,
 0.013480402640998363,
 0.013429406881332397,
 0.013407044000923634,
 0.013372571170330048,
 0.013369800671935082,
 0.01333269838243723,
 0.013328526541590691,
 0.013282898440957069,
 0.013261311426758767,
 0.013238754197955132,
 0.013221934251487256,
 0.013196706287562848,
 0.013184583075344562,
 0.013163331374526023,
 0.013142114505171776,
 0.013129576556384564,
 0.013112151101231574,
 0.013098246678709983,
 0.01307310588657856,
 0.013062296472489833,
 0.013046456202864647,
 0.013035134971141815,
 0.0130124431848526,

In [21]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [22]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
(i, char) for char, i in target_token_index.items())

In [23]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [24]:
for seq_index in range(50):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    
    print('---------------')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:',decoded_sentence)
   

---------------
Input sentence: hello
Decoded sentence: I think you can with the but that is not and camerain ther flems crivectoom, bewstallitt for a oniec inct 100 sppect ranes. The fortaking protup co prods that just with dinnol disa. The nive be will ot suncess of your prrelm vinery daliand sp to your pist.

---------------
Input sentence: hello
Decoded sentence: I think you can with the but that is not and camerain ther flems crivectoom, bewstallitt for a oniec inct 100 sppect ranes. The fortaking protup co prods that just with dinnol disa. The nive be will ot suncess of your prrelm vinery daliand sp to your pist.

---------------
Input sentence: hello
Decoded sentence: I think you can with the but that is not and camerain ther flems crivectoom, bewstallitt for a oniec inct 100 sppect ranes. The fortaking protup co prods that just with dinnol disa. The nive be will ot suncess of your prrelm vinery daliand sp to your pist.

---------------
Input sentence: good morning
Decoded sente

---------------
Input sentence: pay using paypal without paypal account
Decoded sentence: I think you can with the but that is not and camerain ther flems crivectoom, bewstallitt for a oniec inct 100 sppect ranes. The fortaking protup co prods that just with dinnol disa. The nive be will ot suntecs or poun corminives on HD V loos.

---------------
Input sentence: use remaining account credits
Decoded sentence: I think you can with the but that is not and camerain ther flems crivectoom, bewstallitt for a oniec inct 100 sppect ranes. The fortaking protup co prods that just with dinnol disa. The nive be will ot suncess of your prrelm vinery daliand sp to your pist.

---------------
Input sentence: must make payment immediately checkout
Decoded sentence: I think you can with the but that is not and camerain ther flems crivectoom, bewstallitt for a oniec inct 100 sppect ranes. The fortaking protup co prods that just with dinnol disa. The nive be will ot suncess of your prrelm vinery daliand

In [25]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

text1 = 'hello'
text2 = 'hello works normally image clear send mexico'

vector1 = text_to_vector(text1)
vector2 = text_to_vector(text2)

cosine = get_cosine(vector1, vector2)

print ('Cosine:', cosine)

Cosine: 0.3779644730092272


In [26]:
tmp_input_texts = set(input_texts)

In [27]:
yes = ('y', 'Y')
no = ('n', 'N')

def errorCorrection():
    if egg.startswith(no):
        # Break out of loop.
        raise StopIteration()
        
    elif egg.startswith(yes):
        question = input("Check your question:\n")
        question_input_vector = text_to_vector(question)
        cosine = 0.0
        for tmp_text in tmp_input_texts:
            question_train_vector = text_to_vector(tmp_text)
            if(get_cosine(question_input_vector,question_train_vector) > cosine):
                cosine = get_cosine(question_input_vector,question_train_vector)
                i = input_texts.index(tmp_text)
        print("Did you mean: ",input_texts[i])
try:
    while True:
        egg = input("Do you want to continue? (y/n): ")
        errorCorrection()
except StopIteration:
    pass
print ('Exit')

    

Do you want to continue? (y/n): n
Exit


In [113]:
yes = ('y','Y')
no = ('n','N')

def chat(question):
    question_input_vector = text_to_vector(question)
    cosine = 0.0
    for tmp_text in tmp_input_texts:
        question_train_vector = text_to_vector(tmp_text)
        if(get_cosine(question_input_vector,question_train_vector) > cosine):
            cosine = get_cosine(question_input_vector,question_train_vector)
            i = input_texts.index(tmp_text)

    input_seq = encoder_input_data[i:i+1]
    decoded_sentence = decode_sequence(input_seq)
    print('SI03BOT:\n',decoded_sentence)

try:
    while True:
        print("TYPE n TO EXIT")
        question = input("Human:\n")
        if(question.lower()=='n'):
            raise StopIteration()
        else:
            chat(question)
except StopIteration:
    pass
print ('Exit')

TYPE n TO EXIT
Human:
n
Exit


In [50]:
# import matplotlib.pyplot as plt
# acc = history.history['acc']
# val_acc = history.history['val_acc']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(len(acc))

# plt.plot(epochs, acc, 'bo', label='Training acc')
# plt.plot(epochs, val_acc, 'b', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.legend()

# plt.figure()

# plt.plot(epochs, loss, 'bo', label='Training loss')
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()

# plt.show()